<a href="https://colab.research.google.com/github/SeryeongLee/-/blob/main/0602_credit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install category_encoders

     |████████████████████████████████| 81kB 7.5MB/s 


In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import os
import re

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from scipy.special import erfc
from sklearn.dummy import DummyRegressor
from sklearn.svm import SVR

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split, cross_validate, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor, VotingRegressor

from lightgbm import LGBMRegressor

import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor

import warnings

from category_encoders.ordinal import OrdinalEncoder

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
train = pd.read_csv('/content/drive/MyDrive/creditcard/open/train.csv')
test = pd.read_csv('/content/drive/MyDrive/creditcard/open/test.csv')

## 기본 변환

#### 1) 0, 1 변환

#### 2) 수 변환

In [5]:
# 음수 -> 양수 변환

train.DAYS_BIRTH = -train.DAYS_BIRTH
test.DAYS_BIRTH = -test.DAYS_BIRTH

train.DAYS_EMPLOYED = -train.DAYS_EMPLOYED
test.DAYS_EMPLOYED = -test.DAYS_EMPLOYED

train.begin_month = -train.begin_month
test.begin_month = -test.begin_month

In [6]:
# 365243 인 값 0으로 바꾸기

train.DAYS_EMPLOYED = train.DAYS_EMPLOYED.replace([-365243],[0])
test.DAYS_EMPLOYED = test.DAYS_EMPLOYED.replace([-365243],[0])

In [7]:
# 360으로 나눠서 나이, 일한 년수 계산

train['EMPLOYED']= train.DAYS_EMPLOYED.apply(lambda x: abs(x/360) )
train['age']= train.DAYS_BIRTH.apply(lambda x: abs(x/360) ).astype(int)
train['begin_month']= train.begin_month.apply(lambda x: abs(x) ).astype(int)

test['EMPLOYED']= test.DAYS_EMPLOYED.apply(lambda x: abs(x/360) )
test['age']= test.DAYS_BIRTH.apply(lambda x: abs(x/360) ).astype(int)
test['begin_month']= test.begin_month.apply(lambda x: abs(x) ).astype(int)

#### 3) 로그변환

In [8]:
tr_it=train['income_total']
tr_it_log = np.log1p(tr_it)
train['income_total']= tr_it_log
te_it=test['income_total']
te_it_log = np.log1p(te_it)
test['income_total']= te_it_log

In [9]:
train['EMPLOYED']= np.log1p(train['EMPLOYED'])
test['EMPLOYED']= np.log1p(test['EMPLOYED'])

#### 4) child_num, family_size 관련 컬럼 추가

In [10]:
train['child_num'].value_counts()

0     18340
1      5386
2      2362
3       306
4        47
5        10
14        3
7         2
19        1
Name: child_num, dtype: int64

In [11]:
# child_num이 14, 19인 행 지우기

idx_child_drop = train[(train['child_num'] == 14)|(train['child_num'] == 19)].index
train = train.drop(idx_child_drop)

In [12]:
# child_num이 family_size보다 큰 행도 지우기

idx_child_drop2 = train[train['family_size'] < train['child_num']].index
train = train.drop(idx_child_drop2)

In [13]:
train['cf_ratio'] = train['child_num'] / train['family_size']
test['cf_ratio'] = test['child_num'] / test['family_size']

# train['if_ratio'] = train['income_total'] / train['family_size']
# test['if_ratio'] = test['income_total'] / test['family_size']

In [14]:
train['family_size'].value_counts()

2.0    14106
1.0     5108
3.0     4632
4.0     2260
5.0      291
6.0       44
7.0        9
9.0        2
Name: family_size, dtype: int64

In [18]:
# family_size > 7 인 것도 지우기
train = train[(train['family_size']<=7)]
train = train.reset_index(drop=True)

#### 5) 결측치 채우기 - unemployed 만들고 nan 처리

In [15]:
def unemployed(data):
  data.loc[(data['DAYS_EMPLOYED'] == 0), 'occyp_type'] = 'unemployed'

unemployed(train)
unemployed(test)

In [16]:
train.fillna('Nan', inplace=True)
test.fillna('Nan',inplace=True)


In [17]:
train.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit,EMPLOYED,age,cf_ratio
0,0,F,N,N,0,12.218500,Commercial associate,Higher education,Married,Municipal apartment,13899,4709,1,0,0,0,Nan,2.0,6,1.0,2.644795,38,0.000000
1,1,F,N,Y,1,12.419170,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,11380,1540,1,0,0,1,Laborers,3.0,5,1.0,1.663505,31,0.333333
2,2,M,Y,Y,0,13.017005,Working,Higher education,Married,House / apartment,19087,4434,1,0,1,0,Managers,2.0,22,2.0,2.589016,53,0.000000
3,3,F,N,Y,0,12.218500,Commercial associate,Secondary / secondary special,Married,House / apartment,15088,2092,1,0,1,0,Sales staff,2.0,37,0.0,1.918555,41,0.000000
4,4,F,Y,Y,0,11.967187,State servant,Higher education,Married,House / apartment,15037,2105,1,0,0,0,Managers,2.0,26,2.0,1.923843,41,0.000000


#### 명목형 변수 처리 - OrdinalEncoder

In [19]:
encoder = OrdinalEncoder()

In [20]:
cate_feature = train.dtypes[train.dtypes == 'object'].index.tolist()
print(len(cate_feature))
print(train.dtypes)

8
index              int64
gender            object
car               object
reality           object
child_num          int64
income_total     float64
income_type       object
edu_type          object
family_type       object
house_type        object
DAYS_BIRTH         int64
DAYS_EMPLOYED      int64
FLAG_MOBIL         int64
work_phone         int64
phone              int64
email              int64
occyp_type        object
family_size      float64
begin_month        int64
credit           float64
EMPLOYED         float64
age                int64
cf_ratio         float64
dtype: object


In [21]:
train[cate_feature] = encoder.fit_transform(train[cate_feature],train['credit'])
test[cate_feature] = encoder.fit_transform(test[cate_feature])

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [22]:
train = train.drop(['index', 'FLAG_MOBIL', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'child_num','cf_ratio'], axis = 1)

In [23]:
train.head()

,gender,car,reality,income_total,income_type,edu_type,family_type,house_type,work_phone,phone,email,occyp_type,family_size,begin_month,credit,EMPLOYED,age
0,1,1,1,12.218500,1,1,1,1,0,0,0,1,2.0,6,1.0,2.644795,38
1,1,1,2,12.419170,1,2,2,2,0,0,1,2,3.0,5,1.0,1.663505,31
2,2,2,2,13.017005,2,1,1,2,0,1,0,3,2.0,22,2.0,2.589016,53
3,1,1,2,12.218500,1,2,1,2,0,1,0,4,2.0,37,0.0,1.918555,41
4,1,2,2,11.967187,3,1,1,2,0,0,0,3,2.0,26,2.0,1.923843,41


In [ ]:
# 간단한 autoencoder 모델 구현

AutoEncoder + NN 구현

* 입출력이 같은 구조
* input data를 잘 복원한다.



In [24]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model

인코더 작성

In [25]:
input_dim = Input(shape=(16,)) # 입력 채널 수: 16

In [26]:
encoder1 = Dense(8, activation = 'relu')(input_dim)
encoder2 = Dense(3, activation = 'relu')(encoder1)

In [27]:
decoder1 = Dense(8, activation='relu')(encoder2)
decoder2 = Dense(16, activation='relu')(decoder1)

모델 작성 및 컴파일

In [28]:
autoencoder = Model(inputs = input_dim, outputs = decoder2)


In [29]:
autoencoder.compile(optimizer = 'adam', loss = 'binary_crossentropy')

In [30]:
train_y = train['credit']
train_x = train.drop("credit", axis = 1)

In [31]:
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y,
                 stratify = train_y, 
                 test_size = 0.2,
                 random_state = 10086
                 )


In [32]:
autoencoder.fit(x_train, x_train, batch_size= 32, shuffle= True, validation_data = (x_test,x_test))

662/662 [==============================] - 2s 2ms/step - loss: -41.8761 - val_loss: -46.1601


In [88]:
train = pd.read_csv('/content/drive/MyDrive/creditcard/open/train.csv')

딥러닝 

In [89]:
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

tf.data를 사용해 입력 파이프라인 만들기

In [90]:
train.fillna('Nan',inplace=True)
test.fillna('Nan',inplace=True)

In [91]:
train.drop('index',axis=1,inplace=True)

딥러닝 모델 구현 위해 train dataset으로 train, test, validation 나누기  
test dataset에는 credit 칼럼이 존재하지 않기 때문에

In [92]:
# 딥러닝 모델 구현 위해 train dataset으로 train, test, val 나누기
temp_train, temp_test = train_test_split(train, test_size = 0.2)
temp_train, temp_val = train_test_split(temp_train, test_size = 0.2)
print(len(temp_train), 'train examples')
print(len(temp_val), 'validation examples')
print(len(temp_test), 'test examples')


16932 train examples
4233 validation examples
5292 test examples


In [93]:
temp_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16932 entries, 6313 to 20398
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gender         16932 non-null  object 
 1   car            16932 non-null  object 
 2   reality        16932 non-null  object 
 3   child_num      16932 non-null  int64  
 4   income_total   16932 non-null  float64
 5   income_type    16932 non-null  object 
 6   edu_type       16932 non-null  object 
 7   family_type    16932 non-null  object 
 8   house_type     16932 non-null  object 
 9   DAYS_BIRTH     16932 non-null  int64  
 10  DAYS_EMPLOYED  16932 non-null  int64  
 11  FLAG_MOBIL     16932 non-null  int64  
 12  work_phone     16932 non-null  int64  
 13  phone          16932 non-null  int64  
 14  email          16932 non-null  int64  
 15  occyp_type     16932 non-null  object 
 16  family_size    16932 non-null  float64
 17  begin_month    16932 non-null  float64
 18  cre

In [94]:
def df_to_dataset(d_train,shuffle=True,batch_size = 32):
  d_train = train.copy()
  labels = d_train.pop('credit')
  ds = tf.data.Dataset.from_tensor_slices((dict(d_train), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size = len(d_train))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds


In [95]:
batch_size = 5
train_ds = df_to_dataset(temp_train, batch_size=batch_size)

In [96]:
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of targets:', label_batch)

Every feature: ['gender', 'car', 'reality', 'child_num', 'income_total', 'income_type', 'edu_type', 'family_type', 'house_type', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'FLAG_MOBIL', 'work_phone', 'phone', 'email', 'occyp_type', 'family_size', 'begin_month']
A batch of targets: tf.Tensor([1. 1. 2. 2. 2.], shape=(5,), dtype=float64)


전처리 레이어의 사용 시연

1. 숫자 열

In [97]:
def get_normalization_layer(name, dataset):
  normalizer = preprocessing.Normalization()
  feature_ds = dataset.map(lambda x, y: x[name])
  normalizer.adapt(feature_ds)
  return normalizer

In [98]:
income_total_col = train_features['income_total']
layer = get_normalization_layer('income_total', train_ds)
layer(income_total_col)

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[ 1.6951354 ],
       [-0.46926785],
       [ 0.36999056],
       [-0.11589588],
       [-0.29258186]], dtype=float32)>

숫자 특성이 많은 경우, 먼저 숫자 특성을 연결하고 단일 normalization 레이어를 사용하는 것이 더 효율적

2. 범주 열

In [99]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens = None):
  if dtype == 'string':
    index = preprocessing.StringLookup(max_tokens=max_tokens)
  else:
    index = preprocessing.IntegerLookup(max_values = max_tokens)
  feature_ds = dataset.map(lambda x, y: x[name])
  index.adapt(feature_ds)
  encoder = preprocessing.CategoryEncoding(max_tokens = index.vocabulary_size())
  feature_ds = feature_ds.map(index)
  encoder.adapt(feature_ds)
  return lambda feature: encoder(index(feature))

In [100]:
gender_col = train_features['gender']
layer = get_category_encoding_layer('gender', train_ds, 'string')
layer(gender_col)

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

사용할 열 선택 - 열 몇개만 이용해서 구현  

In [84]:
temp_train['phone'].value_counts()

0    11946
1     4986
Name: phone, dtype: int64

In [101]:
temp_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16932 entries, 6313 to 20398
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gender         16932 non-null  object 
 1   car            16932 non-null  object 
 2   reality        16932 non-null  object 
 3   child_num      16932 non-null  int64  
 4   income_total   16932 non-null  float64
 5   income_type    16932 non-null  object 
 6   edu_type       16932 non-null  object 
 7   family_type    16932 non-null  object 
 8   house_type     16932 non-null  object 
 9   DAYS_BIRTH     16932 non-null  int64  
 10  DAYS_EMPLOYED  16932 non-null  int64  
 11  FLAG_MOBIL     16932 non-null  int64  
 12  work_phone     16932 non-null  int64  
 13  phone          16932 non-null  int64  
 14  email          16932 non-null  int64  
 15  occyp_type     16932 non-null  object 
 16  family_size    16932 non-null  float64
 17  begin_month    16932 non-null  float64
 18  cre

In [102]:
batch_size = 128
train_ds = df_to_dataset(temp_train, batch_size=batch_size)
val_ds = df_to_dataset(temp_val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(temp_test, shuffle=False, batch_size=batch_size)

In [103]:
all_inputs = []
encoded_features = []

# Numeric features.
for header in ['income_total', 'DAYS_BIRTH','child_num','DAYS_EMPLOYED','FLAG_MOBIL','family_size','begin_month']:
  numeric_col = tf.keras.Input(shape=(1,), name=header)
  normalization_layer = get_normalization_layer(header, train_ds)
  encoded_numeric_col = normalization_layer(numeric_col)
  all_inputs.append(numeric_col)
  encoded_features.append(encoded_numeric_col)

work_phone, phone, email

In [104]:
# Categorical features encoded as integers.
em_col = tf.keras.Input(shape=(1,), name='email', dtype='int64')
encoding_layer = get_category_encoding_layer('email', train_ds, dtype='int64',max_tokens=5)
encoded_em_col = encoding_layer(em_col)
all_inputs.append(em_col)
encoded_features.append(encoded_em_col)

In [105]:
work_phone_col = tf.keras.Input(shape=(1,), name='work_phone', dtype='int64')
encoding_layer = get_category_encoding_layer('work_phone', train_ds, dtype='int64',max_tokens=5)
encoded_work_phone_col = encoding_layer(work_phone_col)
all_inputs.append(work_phone_col)
encoded_features.append(encoded_work_phone_col)

In [106]:
ph_col = tf.keras.Input(shape=(1,), name='phone', dtype='int64')
encoding_layer = get_category_encoding_layer('phone', train_ds, dtype='int64',max_tokens=5)
encoded_ph_col = encoding_layer(ph_col)
all_inputs.append(ph_col)
encoded_features.append(encoded_ph_col)

In [107]:
# Categorical features encoded as string.
categorical_cols = ['gender', 'car', 'reality', 'edu_type','family_type','house_type','occyp_type']
for header in categorical_cols:
  categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
  encoding_layer = get_category_encoding_layer(header, train_ds, dtype='string',
                                               max_tokens=5)
  encoded_categorical_col = encoding_layer(categorical_col)
  all_inputs.append(categorical_col)
  encoded_features.append(encoded_categorical_col)

모델 생성, 컴파일 및 훈련하기

In [108]:
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(32, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(all_inputs, output)
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

모델 훈련하기

In [109]:
model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/functional.py:591: UserWarning: Input dict contained keys ['income_type'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


207/207 [==============================] - 2s 4ms/step - loss: -4.5827 - accuracy: 0.2331
Epoch 2/10
207/207 [==============================] - 1s 4ms/step - loss: -30.9317 - accuracy: 0.2369
Epoch 3/10
207/207 [==============================] - 1s 4ms/step - loss: -84.4094 - accuracy: 0.2369
Epoch 4/10
207/207 [==============================] - 1s 4ms/step - loss: -162.8846 - accuracy: 0.2369
Epoch 5/10
207/207 [==============================] - 1s 4ms/step - loss: -265.7181 - accuracy: 0.2369
Epoch 6/10
207/207 [==============================] - 1s 4ms/step - loss: -391.1990 - accuracy: 0.2369
Epoch 7/10
207/207 [==============================] - 1s 4ms/step - loss: -535.8762 - accuracy: 0.2369
Epoch 8/10
207/207 [==============================] - 1s 4ms/step - loss: -705.3365 - accuracy: 0.2369
Epoch 9/10
207/207 [==============================] - 1s 4ms/step - loss: -892.3000 - accuracy: 0.2369
Epoch 10/10
207/207 [==============================] - 1s 4ms/step - loss: -1101.1121 - 

명목형 변수 처리 - AutoEncoder  
(https://dacon.io/competitions/official/235713/codeshare/2562?page=3&dtype=recent) 참고

결측치 Nan 처리 & 레이블 인코딩


In [ ]:
!pip install kaggler

     |████████████████████████████████| 829kB 7.0MB/s 
  Created wheel for kaggler: filename=Kaggler-0.9.6-cp37-cp37m-linux_x86_64.whl size=2959080 sha256=b22437df30b6d8d8fc6ef1883ddc974fc41881dbbea78d61cf5c1c2147edeafc
  Stored in directory: /root/.cache/pip/wheels/75/06/a9/f2a7b621e4c6b0571cec751c462683e7528d4fd0bb51060b4f
  Created wheel for ml-metrics: filename=ml_metrics-0.1.4-cp37-none-any.whl size=7850 sha256=7c49128d5c6a6bf04876d4a742078272e9af6664255aacfa2454418aae035973
  Stored in directory: /root/.cache/pip/wheels/b3/61/2d/776be7b8a4f14c5db48c8e5451451cabc58dc6aa7ee3801163
Successfully built kaggler ml-metrics


In [ ]:
from kaggler.preprocessing import LabelEncoder
from kaggler.model import AutoLGB

In [ ]:
index_col = 'index'
target_col = 'credit'

In [ ]:
trn = pd.read_csv('/content/drive/MyDrive/creditcard/open/train.csv',index_col=index_col)
tst = pd.read_csv('/content/drive/MyDrive/creditcard/open/test.csv',index_col=index_col)
sub = pd.read_csv('/content/drive/MyDrive/creditcard/open/sample_submission.csv',index_col=index_col)


In [ ]:
df = pd.concat([trn.drop(target_col, axis = 1),tst],axis=0)

In [ ]:
cate_cols = [x for x in df.columns if df[x].dtype == 'object']
num_cols = [x for x in df.columns if x not in cate_cols + [target_col]]
feature_cols = num_cols + cate_cols
print(len(feature_cols), len(cate_cols), len(num_cols))

18 8 10


In [ ]:
lbe = LabelEncoder(min_obs=10)
df[cate_cols] = lbe.fit_transform(df[cate_cols])
df[cate_cols].head()

,gender,car,reality,income_type,edu_type,family_type,house_type,occyp_type
index,,,,,,,,
0,0,0,1,1,1,0,2,0
1,0,0,0,1,0,2,0,1
2,1,1,0,0,1,0,0,4
3,0,0,0,1,0,0,0,3
4,0,1,0,3,1,0,0,4


autoencoder을 명목형 변수에만 해준 후 autolgbm 학습